In [11]:
import pandas as pd
import requests
import re
pd.set_option('display.max_columns', 500)


In [2]:
all_lines = pd.read_csv('datasets/prize_picks_lines.csv')

In [3]:
def call_endpoint(url, max_level=3, include_new_player_attributes=False):
    '''
    takes: 
        - url (str): the API endpoint to call
        - max_level (int): level of json normalizing to apply
        - include_player_attributes (bool): whether to include player object attributes in the returned dataframe
    returns:
        - df (pd.DataFrame): a dataframe of the call response content
    '''
    resp = requests.get(url).json()
    data = pd.json_normalize(resp['data'], max_level=max_level)
    included = pd.json_normalize(resp['included'], max_level=max_level)
    if include_new_player_attributes:
        inc_cop = included[included['type'] == 'new_player'].copy().dropna(axis=1)
        data = pd.merge(data
                        , inc_cop
                        , how='left'
                        , left_on=['relationships.new_player.data.id'
                                   ,'relationships.new_player.data.type']
                        , right_on=['id', 'type']
                        , suffixes=('', '_new_player'))
    return data

In [24]:
url = 'http://partner-api.prizepicks.com/leagues' # leagues that are live on our board
print(url)
leagues_df = call_endpoint(url)

http://partner-api.prizepicks.com/leagues


In [25]:
leagues_df = leagues_df[['attributes.name', 'id', 'attributes.projections_count']]
leagues_df[leagues_df['attributes.projections_count'] > 0]

,attributes.name,id,attributes.projections_count
1,WNBA,3,16
3,NHL2P,234,10
10,NBA,7,393
13,MLB,2,569
15,NHL,8,227
23,MMA,12,19
24,SOCCER,82,574
25,MLBLIVE,231,23
30,COD,145,112
31,NBA1Q,192,12


In [27]:
league_id = 2
url = f'https://partner-api.prizepicks.com/projections?league_id={league_id}&per_page=1000'
prop_df = call_endpoint(url, include_new_player_attributes=True)

In [28]:
simple_prop_df = prop_df.copy()

simple_prop_df = simple_prop_df[['attributes.start_time','attributes.name', 'type', 'id', 'attributes.adjusted_odds', 'attributes.line_score', 'attributes.odds_type', 'attributes.projection_type', 'attributes.stat_type', 'relationships.new_player.data.id','attributes.position', 'attributes.team', 'attributes.image_url']]
simple_prop_df = simple_prop_df.rename(columns={
    'attributes.start_time' : 'game_date',
    'attributes.line_score' : 'prop_line',
    'attributes.odds_type' : 'odds_type',
    'attributes.stat_type' : 'prop_type',
    'relationships.new_player.data.id' : 'PP_ID',
    'attributes.name' : 'Name',
    'attributes.image_url' : 'image_url',
    'attributes.position' : 'position',
    'attributes.team' : 'team_name'
})
simple_prop_df['game_date_time'] = pd.to_datetime(simple_prop_df['game_date'])
simple_prop_df['game_date'] = simple_prop_df['game_date_time'].dt.date


In [29]:


pitcher_props = ['Pitcher Fantasy Score', 'Hits Allowed', 'Pitching Outs', 'Pitcher Strikeouts', 'Earned Runs Allowed', 'Walks Allowed']

simple_prop_df[simple_prop_df['prop_type'].isin(pitcher_props)]

,game_date,Name,type,id,attributes.adjusted_odds,prop_line,odds_type,attributes.projection_type,prop_type,PP_ID,position,team_name,image_url,game_date_time
302,2024-05-11,Ryan Feltner,projection,2434460,None,4.0,standard,Single Stat,Pitcher Strikeouts,69826,P,COL,https://static.prizepicks.com/images/players/m...,2024-05-11 20:10:00-04:00
305,2024-05-11,Andrew Heaney,projection,2434461,None,5.0,standard,Single Stat,Pitcher Strikeouts,125198,P,TEX,https://static.prizepicks.com/images/players/m...,2024-05-11 20:10:00-04:00
328,2024-05-11,Matt Waldron,projection,2434480,None,3.5,standard,Single Stat,Pitcher Strikeouts,160209,P,SD,https://static.prizepicks.com/images/players/p...,2024-05-11 20:40:00-04:00
329,2024-05-11,James Paxton,projection,2434481,None,4.0,standard,Single Stat,Pitcher Strikeouts,189510,P,LAD,https://static.prizepicks.com/images/players/m...,2024-05-11 20:40:00-04:00
330,2024-05-11,Matt Waldron,projection,2435608,None,5.5,standard,Single Stat,Hits Allowed,160209,P,SD,https://static.prizepicks.com/images/players/p...,2024-05-11 20:40:00-04:00
358,2024-05-11,Tyler Anderson,projection,2434704,None,5.0,standard,Single Stat,Pitcher Strikeouts,125310,P,LAA,https://static.prizepicks.com/images/players/m...,2024-05-11 21:38:00-04:00
359,2024-05-11,Cole Ragans,projection,2434705,None,7.5,standard,Single Stat,Pitcher Strikeouts,147613,P,KC,https://static.prizepicks.com/images/players/m...,2024-05-11 21:38:00-04:00
376,2024-05-11,Bryce Miller,projection,2435333,None,6.5,standard,Single Stat,Pitcher Strikeouts,140647,P,SEA,https://static.prizepicks.com/images/players/p...,2024-05-11 21:40:00-04:00
377,2024-05-11,Joey Estes,projection,2435334,None,5.0,standard,Single Stat,Pitcher Strikeouts,161576,P,OAK,https://static.prizepicks.com/images/players/p...,2024-05-11 21:40:00-04:00
378,2024-05-11,Joey Estes,projection,2435610,None,4.5,standard,Single Stat,Hits Allowed,161576,P,OAK,https://static.prizepicks.com/images/players/p...,2024-05-11 21:40:00-04:00


In [30]:
all_lines = pd.concat([all_lines, simple_prop_df])
all_lines = all_lines.drop_duplicates(subset=['game_date_time', 'Name', 'prop_type', 'prop_line'], keep='first')


In [31]:
all_lines

,game_date,Name,type,id,attributes.adjusted_odds,prop_line,odds_type,attributes.projection_type,prop_type,PP_ID,position,team_name,image_url,game_date_time
0,2024-05-10,Framber Valdez,projection,2429175,NaN,33.5,standard,Single Stat,Pitcher Fantasy Score,67725,P,HOU,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
1,2024-05-10,Yordan Alvarez,projection,2429695,True,0.5,demon,Single Stat,Home Runs,67700,OF,HOU,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
2,2024-05-10,Jared Jones,projection,2429389,NaN,6.5,standard,Single Stat,Pitcher Strikeouts,192811,P,PIT,https://static.prizepicks.com/images/players/p...,2024-05-10 18:40:00-04:00
3,2024-05-10,Javier Assad,projection,2429798,NaN,15.5,standard,Single Stat,Pitching Outs,73818,P,CHC,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
4,2024-05-10,Nico Hoerner,projection,2429712,True,0.5,demon,Single Stat,Stolen Bases,69425,IF,CHC,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2024-05-11,Cole Ragans,projection,2435685,True,8.5,demon,Single Stat,Pitcher Strikeouts,147613,P,KC,https://static.prizepicks.com/images/players/m...,2024-05-11 21:38:00-04:00
565,2024-05-11,Tyler Anderson,projection,2435686,True,3.5,goblin,Single Stat,Pitcher Strikeouts,125310,P,LAA,https://static.prizepicks.com/images/players/m...,2024-05-11 21:38:00-04:00
566,2024-05-11,Bryce Miller,projection,2435687,True,5.5,goblin,Single Stat,Pitcher Strikeouts,140647,P,SEA,https://static.prizepicks.com/images/players/p...,2024-05-11 21:40:00-04:00
567,2024-05-11,Bryce Miller,projection,2435688,True,7.5,demon,Single Stat,Pitcher Strikeouts,140647,P,SEA,https://static.prizepicks.com/images/players/p...,2024-05-11 21:40:00-04:00


In [32]:
all_lines.to_csv('datasets/prize_picks_lines.csv', index=False)

In [4]:
league_id = 231
url = f'https://partner-api.prizepicks.com/projections?league_id={league_id}&per_page=1000'
prop_df = call_endpoint(url, include_new_player_attributes=True)

In [9]:
simple_prop_df = prop_df.copy()

simple_prop_df = simple_prop_df[['attributes.start_time','attributes.name', 'attributes.description','type', 'id', 'attributes.adjusted_odds', 'attributes.line_score', 'attributes.odds_type', 'attributes.projection_type', 'attributes.stat_type', 'relationships.new_player.data.id','attributes.position', 'attributes.team', 'attributes.image_url']]
simple_prop_df = simple_prop_df.rename(columns={
    'attributes.start_time' : 'game_date',
    'attributes.line_score' : 'prop_line',
    'attributes.odds_type' : 'odds_type',
    'attributes.stat_type' : 'prop_type',
    'relationships.new_player.data.id' : 'PP_ID',
    'attributes.name' : 'Name',
    'attributes.image_url' : 'image_url',
    'attributes.position' : 'position',
    'attributes.team' : 'team_name'
})
simple_prop_df['game_date_time'] = pd.to_datetime(simple_prop_df['game_date'])
simple_prop_df['game_date'] = simple_prop_df['game_date_time'].dt.date


In [12]:
def extract_numbers(row):
    string = row['attributes.description']
    # Use regular expression to find all numbers in the string
    numbers = re.findall(r'\d+', string)
    
    # Convert the extracted strings to integers and return as a list
    return [int(num) for num in numbers]

simple_prop_df['Innings'] = simple_prop_df.apply(extract_numbers, axis=1)

In [13]:
simple_prop_df

,game_date,Name,attributes.description,type,id,attributes.adjusted_odds,prop_line,odds_type,attributes.projection_type,prop_type,PP_ID,position,team_name,image_url,game_date_time,Innings
0,2024-05-18,MacKenzie Gore,PHI 3rd Inning,projection,2480061,None,0.5,standard,Single Stat,Hits Allowed,127526,P,WSH,https://static.prizepicks.com/images/players/m...,2024-05-18 19:35:00-04:00,[3]
1,2024-05-18,Cristopher Sánchez,WSH 4th Inning,projection,2475936,None,15.5,standard,Single Stat,Pitches Thrown,143041,P,PHI,https://static.prizepicks.com/images/players/m...,2024-05-18 19:35:00-04:00,[4]
2,2024-05-18,MacKenzie Gore,PHI 3+4 Innings,projection,2480082,None,0.5,standard,Single Stat,Runs Allowed,127526,P,WSH,https://static.prizepicks.com/images/players/m...,2024-05-18 19:35:00-04:00,"[3, 4]"
3,2024-05-18,MacKenzie Gore,PHI 3+4+5 Innings,projection,2479315,None,3.5,standard,Single Stat,Pitcher Strikeouts,127526,P,WSH,https://static.prizepicks.com/images/players/m...,2024-05-18 19:35:00-04:00,"[3, 4, 5]"
4,2024-05-18,MacKenzie Gore,PHI 3rd Inning,projection,2475937,None,16.5,standard,Single Stat,Pitches Thrown,127526,P,WSH,https://static.prizepicks.com/images/players/m...,2024-05-18 19:35:00-04:00,[3]
5,2024-05-18,Cristopher Sánchez,WSH 4th Inning,projection,2480065,None,0.5,standard,Single Stat,Hits Allowed,143041,P,PHI,https://static.prizepicks.com/images/players/m...,2024-05-18 19:35:00-04:00,[4]
6,2024-05-18,Logan Allen,MIN 4th Inning,projection,2480096,None,0.5,standard,Single Stat,Hits Allowed,140494,P,CLE,https://static.prizepicks.com/images/players/p...,2024-05-18 19:40:00-04:00,[4]
7,2024-05-18,Logan Allen,MIN 4+5 Innings,projection,2480102,None,0.5,standard,Single Stat,Runs Allowed,140494,P,CLE,https://static.prizepicks.com/images/players/p...,2024-05-18 19:40:00-04:00,"[4, 5]"
8,2024-05-18,Logan Allen,MIN 4th Inning,projection,2475944,None,15.5,standard,Single Stat,Pitches Thrown,140494,P,CLE,https://static.prizepicks.com/images/players/p...,2024-05-18 19:40:00-04:00,[4]
9,2024-05-18,Logan Allen,MIN 4+5 Innings,projection,2479324,None,1.5,standard,Single Stat,Pitcher Strikeouts,140494,P,CLE,https://static.prizepicks.com/images/players/p...,2024-05-18 19:40:00-04:00,"[4, 5]"
